In [1]:
!pip install tensorflow-text --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 21.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import models,layers
import tensorflow_text as text

In [3]:
text_input = layers.Input(shape=(),dtype=tf.string)
prep = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = prep(text_input)

encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"] #[batchsize,512]
sequence_ouptut = outputs["sequence_output"] #[batchsize,seqlength,512]

In [5]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["My name is Anthony and I am doing today, I am in Dubai"])
print(embedding_model(sentences).shape)

(1, 512)


In [6]:
embedding_model2 = tf.keras.Model(text_input, sequence_ouptut)
sentences = tf.constant(["My name is anshu here, who are you"])
print(embedding_model2(sentences).shape)

(1, 128, 512)


In [ ]:
sentences.shape

TensorShape([1])

In [7]:
doc1 = ["India"]
doc2 = ["I love coding"]
doc3 = ["I love python porgamming and Data Science. Machine Learning is amazing."]
data = [doc1,doc2,doc3]
for doc in data:
  doc = tf.constant(doc)
  print(embedding_model(doc).shape)

(1, 512)
(1, 512)
(1, 512)


In [8]:
for doc in data:
  doc = tf.constant(doc)
  print(embedding_model2(doc).shape)

(1, 128, 512)
(1, 128, 512)
(1, 128, 512)


# Text Classification

In [ ]:
# Text Classification

In [9]:
!wget -q https://www.dropbox.com/s/p1z32pkhs9j7cv3/test_data.txt
!wget -q https://www.dropbox.com/s/ed3qcu1231e8ubx/train_5500.txt

In [10]:
train_data = open("train_5500.txt").readlines()
test_data = open("test_data.txt").readlines()

In [11]:
len(train_data)

5452

In [12]:
len(test_data)

500

In [13]:
train_data[:5]

['DESC:manner How did serfdom develop in and then leave Russia ?\n',
 'ENTY:cremat What films featured the character Popeye Doyle ?\n',
 "DESC:manner How can I find a list of celebrities ' real names ?\n",
 'ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?\n',
 'ABBR:exp What is the full form of .com ?\n']

In [14]:
def prepare_data(data):
  x = []
  y = []
  for doc in data:
    docs = doc.split(" ")
    y.append(docs[0].split(":")[0])
    x.append(" ".join(docs[1:]).strip())
  return x,y


In [15]:
xtrain,ytrain = prepare_data(train_data)
xtest,ytest = prepare_data(test_data)

In [16]:
import pandas as pd

In [17]:
ytrain = pd.DataFrame(ytrain)
ytrain = pd.get_dummies(ytrain)
ytrain.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,1,0,0,0,0,0


In [18]:
ytest = pd.DataFrame(ytest)
ytest = pd.get_dummies(ytest)
ytest.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,0,0,0,1


In [19]:
categories = list(ytrain.columns)
categories = [i[2:] for i in categories]
categories

['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']

### Text Classification using Transfer learning

In [23]:
# create a function which will accept a text doc and return the corresponding vector
def sent2vec(doc):
  vector = embedding_model(tf.squeeze(tf.cast(doc,tf.string),axis=1))
  return vector

In [24]:
from tensorflow.keras import models,layers

In [25]:
input_layer = layers.Input(shape=(1,),dtype='string')
embedding_layer = layers.Lambda(sent2vec,output_shape=(512,))(input_layer)
dense1 = layers.Dense(256,activation='relu')(embedding_layer)
output_layer = layers.Dense(6,activation='softmax')(dense1)

model = models.Model(inputs=[input_layer],outputs=output_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'pooler_transform/bias:0' shape=(512,) dtype=float32>
  <tf.Variable 'pooler_transform/kernel:0' shape=(512, 512) dtype=float32>
  <tf.Variable 'transformer/layer_3/output_layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/output_layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/output/bias:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/output/kernel:0' shape=(2048, 512) dtype=float32>
  <tf.Variable 'transformer/layer_3/intermediate/bias:0' shape=(2048,) dtype=float32>
  <tf.Variable 'transformer/layer_3/intermediate/kernel:0' shape=(512, 2048) dtype=float32>
  <tf.Variable 'transformer/layer_3/self_attention_layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/self_attention_layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'transfo

In [26]:
import numpy as np
xtrain = np.array(xtrain).reshape(-1,1)
xtest = np.array(xtest).reshape(-1,1)
print(xtrain.shape)
print(xtest.shape)

(5452, 1)
(500, 1)


In [27]:
model.fit(xtrain,ytrain,epochs=20,batch_size=64,validation_data=(xtest,ytest))

Epoch 1/20
86/86 [==============================] - 62s 302ms/step - loss: 1.2779 - accuracy: 0.4932 - val_loss: 0.9145 - val_accuracy: 0.6800
Epoch 2/20
86/86 [==============================] - 26s 298ms/step - loss: 1.0352 - accuracy: 0.6042 - val_loss: 0.6762 - val_accuracy: 0.7600
Epoch 3/20
86/86 [==============================] - 26s 305ms/step - loss: 0.9600 - accuracy: 0.6339 - val_loss: 0.6438 - val_accuracy: 0.7860
Epoch 4/20
86/86 [==============================] - 26s 299ms/step - loss: 0.9452 - accuracy: 0.6381 - val_loss: 0.5950 - val_accuracy: 0.8000
Epoch 5/20
86/86 [==============================] - 26s 298ms/step - loss: 0.9027 - accuracy: 0.6570 - val_loss: 0.5781 - val_accuracy: 0.8320
Epoch 6/20
86/86 [==============================] - 26s 305ms/step - loss: 0.8865 - accuracy: 0.6605 - val_loss: 0.6542 - val_accuracy: 0.7120
Epoch 7/20
86/86 [==============================] - 26s 300ms/step - loss: 0.8535 - accuracy: 0.6792 - val_loss: 0.5625 - val_accuracy: 0.8200

In [28]:
new_doc = ["Who is president of United States of America?","What is price for Harley Davidson?"]
new_doc = np.array(new_doc).reshape(-1,1)
output = model.predict(new_doc)

1/1 [==============================] - 0s 427ms/step


In [29]:
output2 = [np.argmax(i) for i in output]
output2 = [categories[i] for i in output2]
output2

['HUM', 'DESC']